In [5]:
import pandas as pd
import numpy as np
import os

In [2]:
all_categories = "Arts and Entertainment·Cars & Other Vehicles·Computers and Electronics·Education and Communications·Family Life·Finance and Business·Food and Entertaining·Health·Hobbies and Crafts·Holidays and Traditions·Home and Garden·Personal Care and Style·Pets and Animals·Philosophy and Religion·Relationships·Sports and Fitness·Travel·Work World·Youth"
all_categories = all_categories.lower()
all_categories = all_categories.split("·")

df = pd.read_csv("data/wikihowSep.csv")
df = df[df.sectionLabel != "Steps"]
df["title"] = df["title"].str.lower()
df["sectionLabel"] = df["sectionLabel"].str.lower()

wikiCat = pd.read_csv("data/cate.csv", sep=",", error_bad_lines=False, names=["title", "category"])
wikiCat['title'] = wikiCat['title'].str.replace("https://www.wikihow.com/", "").str.replace("%22", "").str.replace(
    "-", " ").str.lower()
wikiCat['title'] = ["how to " + i for i in wikiCat['title'].tolist()]
wikiCat['category'] = wikiCat['category'].str.lower()

youth


In [169]:
all_categories

['arts and entertainment',
 'cars & other vehicles',
 'computers and electronics',
 'education and communications',
 'family life',
 'finance and business',
 'food and entertaining',
 'health',
 'hobbies and crafts',
 'holidays and traditions',
 'home and garden',
 'personal care and style',
 'pets and animals',
 'philosophy and religion',
 'relationships',
 'sports and fitness',
 'travel',
 'work world',
 'youth']

In [177]:
cat = "sports and fitness"

In [178]:
tasks = wikiCat[wikiCat.category.str.contains(cat)].title.tolist()

# Uniformly Sample negative queries from any task

In [278]:
for cat in all_categories:
    tasks, queries = [], []
    cat = cat.replace(" ", "_")
#     print(cat)
    folder = "csv/%s/" % cat
    files = []
    for file in os.listdir(folder):
        if os.stat(folder + file).st_size > 0:
            if ".csv" in file:
                task_name = file.split(".csv")[0].replace("_", " ")
                tmp = pd.read_csv("csv/%s/%s" %(cat,file), names=["q", "qq"], sep=";", header=0)
                try:
                    query = tmp.q.str.lower().str.replace("?", "").unique().tolist() + tmp.qq.str.lower().str.replace("?", "").unique().tolist()
                except:
#                   no sub queries
                    query = tmp.q.str.lower().str.replace("?", "").unique().tolist()
                queries.extend(query)
                tasks.extend([task_name]*len(query))
                files.append(task_name)
    print(len(files))
#     break
            
#     #   generate query-task pair dataset
#     x_task, x_query, label = [], [], []
#     for t, pq, nq in zip(tasks, queries, queries[::-1]):
#         x_query.append(pq)
#         x_query.append(nq)
#         x_task.extend([t, t])
#         label.extend([1,0])
#     _df = pd.DataFrame.from_dict({'task': x_task, "query": x_query, "label": label})
#     _df.to_csv("data/query_task_prediction/%s.csv" % cat, index=False, header=False)

4475
1864
7377
9477
2426
11722
3392
13256
6032
628
6446
5224
7246
841
3853
3170
836
2549
4018


# Sample negative queries within the same task

In [274]:
def read_scraper_file(df):
    try:
        query = df.q.str.lower().str.replace("?", "").unique().tolist() + df.qq.str.lower().str.replace("?", "").unique().tolist()
    except:
#       no sub queries
        query = df.q.str.lower().str.replace("?", "").unique().tolist()
    return query

for cat in all_categories:
    tasks = wikiCat[wikiCat.category.str.contains(cat)].title.tolist()
    sdf = df[df.title.isin(tasks)].drop_duplicates(["sectionLabel"])
    subtasks = df[df.title.isin(tasks)].drop_duplicates(["sectionLabel"]).sectionLabel.unique().tolist()

    cat = cat.replace(" ", "_")
    print(cat)
    folder = "csv/%s/" % cat
    scraper_subtasks = []
    for file in os.listdir(folder):
        if os.stat(folder + file).st_size > 0:
            if ".csv" in file:
                task_name = file.split(".csv")[0].replace("_", " ")
#                 print(task_name)
                if task_name in subtasks:
                    scraper_subtasks.append(task_name)
            
    x_task, x_query, label = [], [], []
    for  name, row in sdf.groupby("title"):
        check = set.intersection(set(row.sectionLabel.tolist()), set(scraper_subtasks))
        if len(check) > 1:
            check = list(check)
            for i in range(len(check)):
                subtask_name = check[i]
                _df = pd.read_csv("csv/%s/%s.csv" %(cat,subtask_name.replace(" ", "_")), names=["q", "qq"], sep=";", header=0)
                pos_query = read_scraper_file(_df)

                neg_query_pool = []
                for _subtask_name in check[:i] + check[i+1:]:
                    _df = pd.read_csv("csv/%s/%s.csv" %(cat, _subtask_name.replace(" ", "_")), names=["q", "qq"], sep=";", header=0)
                    neg_query_pool.extend(read_scraper_file(_df))
                
                if len(pos_query) <= len(neg_query_pool):
                    neg_query = random.sample(neg_query_pool, len(pos_query))
                else:
                    neg_query = []
                    while len(neg_query) < len(pos_query):
                        
                        neg_query.append(random.sample(neg_query_pool, 1))
#                     print(len(neg_query_pool), len(pos_query))
#                     neg_query = neg_query_pool + random.sample(neg_query_pool, len(pos_query) - len(neg_query_pool))
                
                for pq, nq in zip(pos_query, neg_query):
                    x_query.append(pq)
                    x_query.append(nq)
                    x_task.extend([subtask_name, subtask_name])
                    label.extend([1,0])
    _df = pd.DataFrame.from_dict({'task': x_task, "query": x_query, "label": label})
    _df.to_csv("data/query_task_prediction/same_task/%s.csv" % cat, index=False, header=False)
            




arts_and_entertainment
cars_&_other_vehicles
computers_and_electronics
education_and_communications
family_life
finance_and_business
food_and_entertaining
health
hobbies_and_crafts
holidays_and_traditions
home_and_garden
personal_care_and_style
pets_and_animals
philosophy_and_religion
relationships
sports_and_fitness
travel
work_world
youth


# question ranking data

In [345]:
for cat in all_categories:
    tasks = wikiCat[wikiCat.category.str.contains(cat)].title.tolist()
    sdf = df[df.title.isin(tasks)].drop_duplicates(["sectionLabel"])
    subtasks = df[df.title.isin(tasks)].drop_duplicates(["sectionLabel"]).sectionLabel.unique().tolist()

    cat = cat.replace(" ", "_")
    print(cat)
    folder = "csv/%s/" % cat
    scraper_subtasks = []
    for file in os.listdir(folder):
        if os.stat(folder + file).st_size > 0:
            if ".csv" in file:
                task_name = file.split(".csv")[0].replace("_", " ")
#                 print(task_name)
                if task_name in subtasks:
                    scraper_subtasks.append(task_name)
            
    x_task, x_query1, x_query2, label1, label2 = [], [], [], [], []
    for  task_name, row in sdf.groupby("title"):
        check = set.intersection(set(row.sectionLabel.tolist()), set(scraper_subtasks))
        if len(check) > 1:
            check = list(check)
            for i in range(len(check) - 1):
                subtask_name1 = check[i]
                subtask_name2 = check[i+1]
                _df1 = pd.read_csv("csv/%s/%s.csv" %(cat,subtask_name1.replace(" ", "_")), names=["q", "qq"], sep=";", header=0)
                _df2 = pd.read_csv("csv/%s/%s.csv" %(cat,subtask_name2.replace(" ", "_")), names=["q", "qq"], sep=";", header=0)
                
                pos_query = read_scraper_file(_df1)
                neg_query_pool = read_scraper_file(_df2)
                
                _pos_query, _neg_query = [], []
                for pq in pos_query:
                    for nq in neg_query_pool:
                        _pos_query.append(pq)
                        _neg_query.append(nq)
                
                for pq, nq in zip(_pos_query, _neg_query):
#                   switch order label
                    rand = np.random.randint(2)
                    x_query1.append(pq if rand else nq)
                    x_query2.append(nq if rand else pq)
                    x_task.append(task_name)
                    label1.append(1 if rand else 0)
                    label2.append(0 if rand else 1)
    _df = pd.DataFrame.from_dict({'task': x_task, "query1": x_query1, "query2": x_query2, "label1": label1, "label2": label2})
    _df.to_csv("data/ranking/questions/%s.csv" % cat, index=False, header=False)
#     break
            


arts_and_entertainment
cars_&_other_vehicles
computers_and_electronics
education_and_communications
family_life
finance_and_business
food_and_entertaining
health
hobbies_and_crafts
holidays_and_traditions
home_and_garden
personal_care_and_style
pets_and_animals
philosophy_and_religion
relationships
sports_and_fitness
travel
work_world
youth


In [346]:
len(_df)

331466

# subtask ranking data

In [339]:
for cat in all_categories:
    tasks = wikiCat[wikiCat.category.str.contains(cat)].title.tolist()
    sdf = df[df.title.isin(tasks)].drop_duplicates(["sectionLabel"])
    subtasks = df[df.title.isin(tasks)].drop_duplicates(["sectionLabel"]).sectionLabel.unique().tolist()

    cat = cat.replace(" ", "_")
    print(cat)
    folder = "csv/%s/" % cat
    scraper_subtasks = []
    for file in os.listdir(folder):
        if os.stat(folder + file).st_size > 0:
            if ".csv" in file:
                task_name = file.split(".csv")[0].replace("_", " ")
#                 print(task_name)
                if task_name in subtasks:
                    scraper_subtasks.append(task_name)
            
    x_task, x_query1, x_query2, label1, label2 = [], [], [], [], []
    for  task_name, row in sdf.groupby("title"):
        check = set.intersection(set(row.sectionLabel.tolist()), set(scraper_subtasks))
        if len(check) > 1:
            check = list(check)
            for i in range(len(check) - 1):
                subtask_name1 = check[i]
                subtask_name2 = check[i+1]
                
                rand = np.random.randint(2)
                x_query1.append(subtask_name1 if rand else subtask_name2)
                x_query2.append(subtask_name2 if rand else subtask_name1)
                x_task.append(task_name)
                label1.append(1 if rand else 0)
                label2.append(0 if rand else 1)
    _df = pd.DataFrame.from_dict({'task': x_task, "query1": x_query1, "query2": x_query2, "label1": label1, "label2": label2})
    _df.to_csv("data/ranking/subtasks/%s.csv" % cat, index=False, header=False)            


arts_and_entertainment
cars_&_other_vehicles
computers_and_electronics
education_and_communications
family_life
finance_and_business
food_and_entertaining
health
hobbies_and_crafts
holidays_and_traditions
home_and_garden
personal_care_and_style
pets_and_animals
philosophy_and_religion
relationships
sports_and_fitness
travel
work_world
youth


In [356]:
df

,overview,headline,text,sectionLabel,title
1,"If you want to be well-read, then, in the wor...",\nRead the classics before 1600.,Reading the classics is the very first thing ...,reading the classics,how to be well read
10,Stage names are used by all types of performe...,\nUnderstand what a stage name can do for you.,A stage name can help you accomplish a number...,choosing a stage name,how to pick a stage name1
15,Your dream of becoming a famous artist may no...,\nPractice.,Being called by the muse is a wonderful thing...,building your skills,how to become a famous artist
17,"If you want to be well-read, then, in the wor...",\nRead the classics from 1600-1913.,Though a large amount of material is covered ...,reading the classics,how to be well read
18,"If you want to be well-read, then, in the wor...",\nRead the classics from 1914-1995.,This time period spans the advent of Modernis...,reading the classics,how to be well read
19,"If you want to be well-read, then, in the wor...",\nRead more contemporary classics from 1980 to...,Though these books haven't stood the test of ...,reading the classics,how to be well read
20,"If you want to be well-read, then, in the wor...",\nRead short stories.,Short stories are an incredible genre all the...,becoming well-read in different genres,how to be well read
21,"If you want to be well-read, then, in the wor...",\nRead plays.,"If you want to be well-read, then you also ha...",becoming well-read in different genres,how to be well read
22,"If you want to be well-read, then, in the wor...",\nRead poetry.,Though the people around you may be less like...,becoming well-read in different genres,how to be well read
23,"If you want to be well-read, then, in the wor...",\nRead non-fiction.,"If you really want to be well-read, then you ...",becoming well-read in different genres,how to be well read


In [358]:
df = pd.read_csv("data/wikihowSep.csv")
df = df[df.sectionLabel != "Steps"]
df["title"] = df["title"].str.lower()
df["sectionLabel"] = df["sectionLabel"].str.lower()

In [359]:
dff = pd.read_csv("data/wikihowSep.csv")


In [377]:
df[df.title.str.contains("girlfriend", na=False)]

,overview,headline,text,sectionLabel,title
1421,"If your girlfriend tells you she's pregnant, ...",\nListen to your girlfriend.,When your girlfriend tells you that she’s pre...,reacting to the news,how to react when your girlfriend tells you sh...
1460,"If your girlfriend tells you she's pregnant, ...",\nStay calm.Take a deep breath before respondi...,It's ok to tell your girlfriend what you are ...,reacting to the news,how to react when your girlfriend tells you sh...
1461,"If your girlfriend tells you she's pregnant, ...",\nLet her know you care.Assuming that you will...,Saying something reassuring like “This is rea...,reacting to the news,how to react when your girlfriend tells you sh...
1462,"If your girlfriend tells you she's pregnant, ...",\nKeep in contact with your girlfriend.,If you want to be supportive of your girlfrie...,reacting to the news,how to react when your girlfriend tells you sh...
1463,"If your girlfriend tells you she's pregnant, ...",\nMake sure she is pregnant.,As soon as your girlfriend thinks she is preg...,reacting to the news,how to react when your girlfriend tells you sh...
1464,"If your girlfriend tells you she's pregnant, ...",\nLearn about your options.Your girlfriend may...,"Before making a decision about what to do, ma...",making decisions,how to react when your girlfriend tells you sh...
1465,"If your girlfriend tells you she's pregnant, ...",\nBe patient.Deciding how to handle a pregnanc...,"Give your girlfriend time to think, and don’t...",making decisions,how to react when your girlfriend tells you sh...
1466,"If your girlfriend tells you she's pregnant, ...",\nTalk about it.Maybe your girlfriend knows ex...,"On the other hand, she might be confused, uns...",making decisions,how to react when your girlfriend tells you sh...
1467,"If your girlfriend tells you she's pregnant, ...",\nHelp your girlfriend tell others.,If your girlfriend wants to tell other people...,making decisions,how to react when your girlfriend tells you sh...
1468,"If your girlfriend tells you she's pregnant, ...",\nAsk for help.,"There are many family planning centers, with ...",making decisions,how to react when your girlfriend tells you sh...
